<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/field_sim_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
field_sim_app.py — End-to-end single-script 2D scalar field simulator

Features:
- Leapfrog integration with optional damping and perturbations
- Energy computation (kinetic, gradient, potential) and CSV logging
- Static frame saving (PNG) or real-time animation
- Optional video export (MP4 via ffmpeg or GIF via Pillow)
- Notebook/Colab-safe CLI (ignores unknown args like -f <.json>)
"""

import os
import sys
import math
import csv
import argparse
from pathlib import Path
import numpy as np


# =========================
# Defaults (config section)
# =========================

DEFAULTS = dict(
    nx=256,
    ny=256,
    dx=1.0,
    dt=0.1,
    steps=1000,
    c=1.0,
    lam=1.0,   # lambda is a keyword in Python; use lam
    v=1.0,
    eta=0.0,   # damping
    seed=42,
    init="random",   # random | blob | bias
    random_amp=0.01,
    blob_sigma=20.0,
    bias_value=0.9,
    save_every=0,    # 0 => animate mode; >0 => save frames every N steps
    out_dir="frames",
    cmap="seismic",
    perturb_mag=0.01,
    perturb_prob=0.001,
)


# =========================
# Utilities
# =========================

def ensure_dir(path: str | Path):
    Path(path).mkdir(parents=True, exist_ok=True)


def five_point_laplacian(phi: np.ndarray, dx: float) -> np.ndarray:
    return (
        np.roll(phi, +1, axis=0) +
        np.roll(phi, -1, axis=0) +
        np.roll(phi, +1, axis=1) +
        np.roll(phi, -1, axis=1) -
        4.0 * phi
    ) / (dx * dx)


def central_gradients(phi: np.ndarray, dx: float) -> tuple[np.ndarray, np.ndarray]:
    gx = (np.roll(phi, -1, axis=0) - np.roll(phi, +1, axis=0)) / (2.0 * dx)
    gy = (np.roll(phi, -1, axis=1) - np.roll(phi, +1, axis=1)) / (2.0 * dx)
    return gx, gy


def potential(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return 0.25 * lam * (phi * phi - v * v) ** 2


def dV_dphi(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return lam * phi * (phi * phi - v * v)


# =========================
# Simulator
# =========================

class FieldSimulator:
    def __init__(self, nx, ny, dx, dt, c, lam, v, eta, seed,
                 init, random_amp, blob_sigma, bias_value,
                 perturb_mag, perturb_prob):
        self.nx, self.ny = nx, ny
        self.dx, self.dt = dx, dt
        self.c, self.lam, self.v = c, lam, v
        self.eta = eta
        self.perturb_mag = perturb_mag
        self.perturb_prob = perturb_prob

        self.phi = np.zeros((nx, ny), dtype=np.float64)
        self.phi_prev = np.zeros_like(self.phi)

        self._rng = np.random.default_rng(seed)
        self._initialize(init, random_amp, blob_sigma, bias_value)
        self._check_cfl()

    def _check_cfl(self):
        s = self.c * self.dt / self.dx
        smax = 1.0 / math.sqrt(2.0)
        if s > smax:
            print(f"[WARN] CFL likely violated: c*dt/dx = {s:.3f} > {smax:.3f}", file=sys.stderr)
        else:
            print(f"[INFO] CFL OK: c*dt/dx = {s:.3f} ≤ {smax:.3f}")

    def _initialize(self, init, random_amp, blob_sigma, bias_value):
        if init == "random":
            self.phi[:] = random_amp * (self._rng.random((self.nx, self.ny)) - 0.5)
        elif init == "blob":
            i = np.arange(self.nx); j = np.arange(self.ny)
            ii, jj = np.meshgrid(i, j, indexing="ij")
            r2 = (ii - (self.nx - 1)/2.0)**2 + (jj - (self.ny - 1)/2.0)**2
            self.phi[:] = np.exp(-r2 / (2.0 * blob_sigma * blob_sigma))
        elif init == "bias":
            self.phi[:] = bias_value
        else:
            raise ValueError(f"Unknown init '{init}'. Use: random | blob | bias")
        self.phi_prev[:] = self.phi

    def apply_perturbation(self):
        m, p = self.perturb_mag, self.perturb_prob
        if m <= 0.0 or p <= 0.0:
            return
        mask = self._rng.random((self.nx, self.ny)) < p
        kick = m * (2.0 * self._rng.random((self.nx, self.ny)) - 1.0)
        self.phi += mask * kick

    def step(self):
        lap = five_point_laplacian(self.phi, self.dx)
        force = (self.c * self.c) * lap - dV_dphi(self.phi, self.lam, self.v)

        damp = self.eta * self.dt
        phi_new = (2.0 - damp) * self.phi - (1.0 - damp) * self.phi_prev + (self.dt * self.dt) * force

        self.phi_prev, self.phi = self.phi, phi_new
        self.apply_perturbation()


# =========================
# Analyzer
# =========================

def energy_components(phi, phi_prev, dt, dx, c, lam, v):
    vel = (phi - phi_prev) / dt
    Ek = 0.5 * (vel * vel)

    gx, gy = central_gradients(phi, dx)
    Eg = 0.5 * (c * c) * (gx * gx + gy * gy)

    Ep = potential(phi, lam, v)

    cell = dx * dx
    return float(Ek.sum() * cell), float(Eg.sum() * cell), float(Ep.sum() * cell)


def total_energy(phi, phi_prev, dt, dx, c, lam, v):
    Ek, Eg, Ep = energy_components(phi, phi_prev, dt, dx, c, lam, v)
    return Ek + Eg + Ep


# =========================
# Visualization
# =========================

def plot_field(phi, step, out_dir="frames", cmap="seismic"):
    # Import locally to keep script headless-friendly unless needed
    import matplotlib.pyplot as plt

    ensure_dir(out_dir)
    plt.figure(figsize=(6, 6))
    im = plt.imshow(phi.T, cmap=cmap, origin="lower")
    plt.colorbar(im, fraction=0.046, pad=0.04, label="Field φ")
    plt.title(f"Field Configuration at Step {step}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.savefig(str(Path(out_dir) / f"field_{step:06d}.png"), dpi=120)
    plt.close()


def animate_field(simulator: FieldSimulator, steps=1000, interval=50, cmap="seismic",
                  show=True, show_energy=True, energy_every=10):
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    fig, ax = plt.subplots(figsize=(6, 6))
    img = ax.imshow(simulator.phi.T, cmap=cmap, origin="lower", animated=True)
    plt.colorbar(img, ax=ax, label="Field φ")
    title = ax.set_title("Real-Time Field Evolution")
    txt = ax.text(0.02, 0.98, "", transform=ax.transAxes, va="top", ha="left",
                  color="w", fontsize=10, bbox=dict(facecolor="0.1", alpha=0.5, pad=3))

    def update(frame):
        simulator.step()
        img.set_array(simulator.phi.T)
        title.set_text(f"Step {frame + 1}")
        if show_energy and (frame % energy_every == 0 or frame == steps - 1):
            E = total_energy(simulator.phi, simulator.phi_prev,
                             simulator.dt, simulator.dx, simulator.c, simulator.lam, simulator.v)
            txt.set_text(f"E ≈ {E:.6f}")
        return [img, title, txt]

    ani = animation.FuncAnimation(fig, update, frames=steps, interval=interval, blit=True, repeat=False)
    if show:
        plt.show()
    return ani, fig


def save_video(ani, outfile: str, fps: int = 20):
    # Try ffmpeg first, fallback to PillowWriter
    import matplotlib.animation as animation

    out = Path(outfile)
    ensure_dir(out.parent)
    suffix = out.suffix.lower()
    if suffix in {".mp4", ".m4v"}:
        try:
            writer = animation.FFMpegWriter(fps=fps, bitrate=1800)
            ani.save(str(out), writer=writer)
            print(f"[INFO] Saved video: {out}")
            return
        except Exception as e:
            print(f"[WARN] ffmpeg unavailable or failed ({e}); falling back to GIF.")
            # fall through to GIF
    # GIF fallback
    try:
        writer = animation.PillowWriter(fps=fps)
        ani.save(str(out.with_suffix(".gif")), writer=writer)
        print(f"[INFO] Saved GIF: {out.with_suffix('.gif')}")
    except Exception as e:
        print(f"[ERROR] Could not save video/GIF: {e}", file=sys.stderr)


# =========================
# CLI
# =========================

def build_parser():
    p = argparse.ArgumentParser(
        prog="field_sim_app",
        description="2D scalar field simulator: animate or save frames.",
        add_help=True,
    )
    # Grid and physics
    p.add_argument("--nx", type=int, default=DEFAULTS["nx"])
    p.add_argument("--ny", type=int, default=DEFAULTS["ny"])
    p.add_argument("--dx", type=float, default=DEFAULTS["dx"])
    p.add_argument("--dt", type=float, default=DEFAULTS["dt"])
    p.add_argument("--steps", type=int, default=DEFAULTS["steps"])
    p.add_argument("--c", type=float, default=DEFAULTS["c"])
    p.add_argument("--lam", type=float, default=DEFAULTS["lam"])
    p.add_argument("--v", type=float, default=DEFAULTS["v"])
    p.add_argument("--eta", type=float, default=DEFAULTS["eta"])
    p.add_argument("--seed", type=int, default=DEFAULTS["seed"])

    # Initialization
    p.add_argument("--init", choices=["random", "blob", "bias"], default=DEFAULTS["init"])
    p.add_argument("--random-amp", type=float, default=DEFAULTS["random_amp"])
    p.add_argument("--blob-sigma", type=float, default=DEFAULTS["blob_sigma"])
    p.add_argument("--bias-value", type=float, default=DEFAULTS["bias_value"])

    # Mode and I/O
    p.add_argument("--save-every", type=int, default=DEFAULTS["save_every"],
                   help="If >0: save PNG frames every N steps; if 0: animate.")
    p.add_argument("--out-dir", type=str, default=DEFAULTS["out_dir"])
    p.add_argument("--cmap", type=str, default=DEFAULTS["cmap"])

    # Perturbations
    p.add_argument("--perturb-mag", type=float, default=DEFAULTS["perturb_mag"])
    p.add_argument("--perturb-prob", type=float, default=DEFAULTS["perturb_prob"])

    # Energy logging
    p.add_argument("--energy-csv", type=str, default="",
                   help="Path to CSV to log step and total energy. Empty to disable.")
    p.add_argument("--energy-every", type=int, default=10,
                   help="Compute/log energy every N steps (animation HUD also uses this).")

    # Animation/video
    p.add_argument("--interval", type=int, default=50, help="Animation interval in ms.")
    p.add_argument("--no-show", action="store_true", help="Do not display windows (headless).")
    p.add_argument("--video", type=str, default="",
                   help="Output video filename (.mp4 or .gif). Used only in animate mode.")

    return p


def parse_args():
    parser = build_parser()
    # Be notebook/Colab-safe: ignore unknown args like -f <file.json>
    args, unknown = parser.parse_known_args()
    if unknown:
        print(f"[INFO] Ignoring unknown args: {unknown}")
    return args


# =========================
# Runner
# =========================

def run_save_mode(sim: FieldSimulator, steps: int, save_every: int, out_dir: str,
                  cmap: str, energy_csv: str, energy_every: int):
    ensure_dir(out_dir)
    csv_writer = None
    csv_file = None
    try:
        if energy_csv:
            ensure_dir(Path(energy_csv).parent)
            csv_file = open(energy_csv, "w", newline="")
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["step", "E_total", "E_kinetic", "E_gradient", "E_potential"])

        for step in range(steps + 1):
            if step % max(1, save_every) == 0:
                plot_field(sim.phi, step, out_dir=out_dir, cmap=cmap)
            if (energy_csv and (step % max(1, energy_every) == 0)) or (step == steps):
                Ek, Eg, Ep = energy_components(sim.phi, sim.phi_prev, sim.dt, sim.dx, sim.c, sim.lam, sim.v)
                E = Ek + Eg + Ep
                print(f"Step {step:6d} | E={E:.6f} (Ek={Ek:.6f}, Eg={Eg:.6f}, Ep={Ep:.6f})")
                if csv_writer:
                    csv_writer.writerow([step, E, Ek, Eg, Ep])

            if step < steps:
                sim.step()
    finally:
        if csv_file:
            csv_file.close()


def run_animate_mode(sim: FieldSimulator, steps: int, interval: int, cmap: str,
                     show: bool, energy_every: int, video: str):
    ani, fig = animate_field(sim, steps=steps, interval=interval, cmap=cmap,
                             show=not show is False and not False, show_energy=True,
                             energy_every=energy_every)
    if video:
        save_video(ani, video, fps=max(1, int(1000 / max(1, interval))))


def main():
    args = parse_args()

    sim = FieldSimulator(
        nx=args.nx, ny=args.ny, dx=args.dx, dt=args.dt, c=args.c, lam=args.lam, v=args.v,
        eta=args.eta, seed=args.seed, init=args.init, random_amp=args.random_amp,
        blob_sigma=args.blob_sigma, bias_value=args.bias_value,
        perturb_mag=args.perturb_mag, perturb_prob=args.perturb_prob
    )

    if args.save_every and args.save_every > 0:
        # Static saving mode
        if args.no_show:
            # Force headless backend only if user asked; imports are deferred anyway.
            pass
        run_save_mode(sim, steps=args.steps, save_every=args.save_every,
                      out_dir=args.out_dir, cmap=args.cmap,
                      energy_csv=args.energy_csv, energy_every=args.energy_every)
    else:
        # Animation mode
        # Respect --no-show by not opening a window; still can export video if requested.
        run_animate_mode(sim, steps=args.steps, interval=args.interval, cmap=args.cmap,
                         show=not args.no_show, energy_every=args.energy_every,
                         video=args.video)


if __name__ == "__main__":
    main()